In [3]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from GLCM_tf import GLCM

batch_size = 100
num_classes = 10
epochs = 3

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
inputs = Input(shape=(28, 28, 1))
glcm_layer = GLCM(batch_size = 100, symmetric=True, normed=True)(inputs)
#flatten_layer = K.flatten(glcm_layer)
outputs = Dense(10, activation='softmax')(glcm_layer)
model = Model(inputs, outputs)
model.compile(optimizer='Adam', loss="MSE", metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
glcm_2 (GLCM)                (100, 4)                  0         
_________________________________________________________________
dense_2 (Dense)              (100, 10)                 50        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist = model.fit(x_train, y_train,
          batch_size = 100,
          epochs=epochs,
          verbose=1)

Epoch 1/3
60000/60000 [==============================] - 33s - loss: 0.0900 - acc: 0.1114    
Epoch 2/3
 2600/60000 [>.............................] - ETA: 29s - loss: 0.0900 - acc: 0.1073

In [ ]:
print (hist.history)

In [ ]:
type(hist.history)